# Efecto del tamaño de la muestra en validación cruzada

También es importante comprender cómo los diferentes errores están influenciados por el número de muestras disponibles.

In [ ]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing(as_frame=True)
data, target = housing.data, housing.target
target *= 100

In [ ]:
from sklearn.tree import DecisionTreeRegressor

regressor = DecisionTreeRegressor()

## Curva de aprendizaje
Podemos variar el número de muestras en el conjunto de entrenamiento y repetir el experimento. 

Los puntajes de entrenamiento y prueba se pueden trazar de manera similar a la curva de validación, pero en lugar de variar un hiperparámetro, variamos el número de muestras de entrenamiento.

Esta curva se llama **Curva de aprendizaje**.
- Proporciona información sobre el beneficio de agregar nuevas muestras de capacitación para mejorar el rendimiento de generalización de un modelo.

In [ ]:
# Calculemos la curva de aprendizaje para un árbol de decisión y variemos la proporción del entrenamiento establecida del 10% al 100%.

import numpy as np
train_sizes = np.linspace(0.1, 1.0, num=5, endpoint=True)
train_sizes

In [ ]:
# usamos validación cruzada ShuffleSplit para evaluar nuestro modelo predictivo.

from sklearn.model_selection import ShuffleSplit

cv = ShuffleSplit(n_splits=30, test_size=0.2)

In [ ]:
# listos para llevar a cabo el experimento:

from sklearn.model_selection import learning_curve

results = learning_curve(
    regressor, data, target, train_sizes=train_sizes, cv=cv,
    scoring="neg_mean_absolute_error", n_jobs=2)
train_size, train_scores, test_scores = results[:3]

# Convertir el negativo de los puntajes de error
train_errors, test_errors = -train_scores, -test_scores

In [ ]:
# graficar la curva.

import matplotlib.pyplot as plt

plt.errorbar(train_size, train_errors.mean(axis=1),
             yerr=train_errors.std(axis=1), label="Error de entrenamiento")
plt.errorbar(train_size, test_errors.mean(axis=1),
             yerr=test_errors.std(axis=1), label="Error de prueba")
plt.legend()

plt.xscale("log")
plt.xlabel("Número de muestras en el conjunto de entrenamiento")
plt.ylabel("Error absoluto medio (k $)")
plt.title("Curva de aprendizaje para el árbol de decisión")

- Observando solo el **error de entrenamiento**, vemos que tenemos un error de 0 k$.
    - Significa que el modelo entrenado está claramente sobreajustado los datos.

- Observando solo el **error de prueba**, observamos que cuantas más muestras se agregan al conjunto de entrenamiento, menor será el error de prueba.
    - Además, estamos buscando la meseta del error de prueba para el cual ya no hay beneficio de agregar muestras o evaluar la posible ganancia de agregar más muestras al conjunto de capacitación.
    - Si logramos una meseta y agregar nuevas muestras en el conjunto de entrenamiento no reduce el error de prueba, es posible que hayamos alcanzado la **tasa de error Bayes** utilizando el modelo disponible.
        - Usar de un modelo más complejo podría ser la única posibilidad de reducir aún más el error de prueba.

## **Conclusiones**

- Modelo **sobreajusta**:
    - El número de muestras en el "conjunto de entrenamiento" es demasiado pequeño
    - *El error de prueba es mucho más grande que el error de entrenamiento*

- Modelo **subajusta**:
    - Los modelos no pueden capturar la forma del "conjunto de entrenamiento"
    - *Incluso el error de entrenamiento es grande*

**Diferentes familias de modelos** = Diferente complejidad & sesgo inductivo

## **Ejercicio**

Para el dataset "house-prices/full.csv" y usando Ridge: 

- Encuentra la meseta del error de prueba, donde ya no hay beneficios de añadir más datos al entrenamiento.